# 0. Prerequisites

In [3]:
import configuration
import coremltools
import cv2
import glob
import os
import platform
import preprocessor
import random
import statistics
import sys
import time
import viz

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')

from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from PIL import Image

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

preproc = preprocessor.PreProcessor()
config = configuration.Configuration()
vizualizer = viz.Vizualizer()

TensorFlow version: 2.10.0
Python Platform: macOS-12.6-arm64-arm-64bit
Tensor Flow Version: 2.10.0
Keras Version: 2.10.0

Python 3.9.12 (main, Jun  1 2022, 06:34:44) 
[Clang 12.0.0 ]
Pandas 1.5.2
Scikit-Learn 1.1.3
SciPy 1.9.3
GPU is available


In [4]:
# Helper methods
# Ref: https://coremltools.readme.io/docs/introductory-quickstart
def convert_tf_saved_model(in_model_file, out_model_file):
    # Define the input type as image, 
    # set pre-processing parameters to normalize the image 
    # to have its values in the interval [-1,1] 
    # as expected by the mobilenet model
    image_input = coremltools.ImageType(shape=(1, 224, 224, 3,),
                               bias=[-1,-1,-1], scale=1/255.)

    # set class labels
    class_labels = list(config.class_dict.values())
    
    classifier_config = coremltools.ClassifierConfig(class_labels)

    model = load_model(in_model_file)
    
    print("[INFO] converting model")
    coreml_model = coremltools.convert(
        model, 
        inputs=[image_input], 
        classifier_config=classifier_config,
    )    

    print("[INFO] saving model")
    coreml_model.save(out_model_file)

# 1. Data Import

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        config.TRAIN_DATA,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

val_generator = train_datagen.flow_from_directory(
        config.TRAIN_DATA,
        target_size=(224,224),
        batch_size=32,
        class_mode='categorical',
        subset='validation')

imgs, labels = next(train_generator)

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [7]:
imgs.shape

(32, 224, 224, 3)

In [ ]:
# k-fold
kfold = StratifiedKFold(n_splits=N_SPLIT,shuffle=True,random_state=42)